# Dask Bags


Dask Bag implements operations like `map`, `filter`, `groupby` and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

Full API documentation is available here: http://docs.dask.org/en/latest/bag-api.html

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:37514 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 4.14 GB


## Create Random Data

We create a random set of record data and store it to disk as many JSON files.  This will serve as our data for this notebook.

In [3]:
!pip install --user mimesis

     |████████████████████████████████| 2.3MB 4.6MB/s eta 0:00:01


In [4]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/home/ubuntu/dask-examples/data/0.json',
 '/home/ubuntu/dask-examples/data/1.json',
 '/home/ubuntu/dask-examples/data/2.json',
 '/home/ubuntu/dask-examples/data/3.json',
 '/home/ubuntu/dask-examples/data/4.json',
 '/home/ubuntu/dask-examples/data/5.json',
 '/home/ubuntu/dask-examples/data/6.json',
 '/home/ubuntu/dask-examples/data/7.json',
 '/home/ubuntu/dask-examples/data/8.json',
 '/home/ubuntu/dask-examples/data/9.json']

## Read JSON data

Now that we have some JSON data in a file lets take a look at it with Dask Bag and Python JSON module.

In [5]:
!head -n 2 data/0.json

{"age": 23, "name": ["Berry", "Jackson"], "occupation": "Sports Coach", "telephone": "(195) 714-0885", "address": {"address": "229 Ellert Brae", "city": "Mayfield Heights"}, "credit-card": {"number": "2306 7790 5218 6927", "expiration-date": "07/24"}}
{"age": 66, "name": ["Keturah", "English"], "occupation": "Physicist", "telephone": "807.452.0962", "address": {"address": "930 Belles Plaza", "city": "Cleveland Heights"}, "credit-card": {"number": "5534 8118 4456 3001", "expiration-date": "03/25"}}


In [6]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads-8..., npartitions=10>

In [7]:
b.take(2)

({'age': 23,
  'name': ['Berry', 'Jackson'],
  'occupation': 'Sports Coach',
  'telephone': '(195) 714-0885',
  'address': {'address': '229 Ellert Brae', 'city': 'Mayfield Heights'},
  'credit-card': {'number': '2306 7790 5218 6927',
   'expiration-date': '07/24'}},
 {'age': 66,
  'name': ['Keturah', 'English'],
  'occupation': 'Physicist',
  'telephone': '807.452.0962',
  'address': {'address': '930 Belles Plaza', 'city': 'Cleveland Heights'},
  'credit-card': {'number': '5534 8118 4456 3001',
   'expiration-date': '03/25'}})

## Map, Filter, Aggregate

We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [8]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30

({'age': 66,
  'name': ['Keturah', 'English'],
  'occupation': 'Physicist',
  'telephone': '807.452.0962',
  'address': {'address': '930 Belles Plaza', 'city': 'Cleveland Heights'},
  'credit-card': {'number': '5534 8118 4456 3001',
   'expiration-date': '03/25'}},
 {'age': 36,
  'name': ['Mark', 'Baldwin'],
  'occupation': 'Pharmacist',
  'telephone': '(416) 253-0182',
  'address': {'address': '21 Chula Trace', 'city': 'West Valley City'},
  'credit-card': {'number': '4725 0803 2541 8521',
   'expiration-date': '02/25'}})

In [9]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field

('Sports Coach', 'Physicist')

In [10]:
b.count().compute()  # Count total number of records

10000

## Chain computations

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [11]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-ag..., npartitions=1>

As with all lazy Dask collections, we need to call `compute` to actually evaluate our result.  The `take` method used in earlier examples is also like `compute` and will also trigger computation.

In [12]:
result.compute()

[('Haulage Contractor', 17),
 ('Sports Administrator', 16),
 ('Plastics Engineer', 15),
 ('Lorry Driver', 15),
 ('Garden Designer', 14),
 ('Landworker', 14),
 ('Research Analyst', 13),
 ('Television Director', 13),
 ('Ticket Inspector', 13),
 ('Cabinet Maker', 13)]

## Transform and Store

Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses.  For that we can use methods like `to_textfiles` and `json.dumps`, or we can convert to Dask Dataframes and use their storage systems, which we'll see more of in the next section.

In [13]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/home/ubuntu/dask-examples/data/processed.0.json',
 '/home/ubuntu/dask-examples/data/processed.1.json',
 '/home/ubuntu/dask-examples/data/processed.2.json',
 '/home/ubuntu/dask-examples/data/processed.3.json',
 '/home/ubuntu/dask-examples/data/processed.4.json',
 '/home/ubuntu/dask-examples/data/processed.5.json',
 '/home/ubuntu/dask-examples/data/processed.6.json',
 '/home/ubuntu/dask-examples/data/processed.7.json',
 '/home/ubuntu/dask-examples/data/processed.8.json',
 '/home/ubuntu/dask-examples/data/processed.9.json']

## Convert to Dask Dataframes

Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes.  Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.  

However, Dask Dataframes also expect data that is organized as flat columns.  It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.

In [14]:
b.take(1)

({'age': 23,
  'name': ['Berry', 'Jackson'],
  'occupation': 'Sports Coach',
  'telephone': '(195) 714-0885',
  'address': {'address': '229 Ellert Brae', 'city': 'Mayfield Heights'},
  'credit-card': {'number': '2306 7790 5218 6927',
   'expiration-date': '07/24'}},)

In [15]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']   
    }

b.map(flatten).take(1)

({'age': 23,
  'occupation': 'Sports Coach',
  'telephone': '(195) 714-0885',
  'credit-card-number': '2306 7790 5218 6927',
  'credit-card-expiration': '07/24',
  'name': 'Berry Jackson',
  'street-address': '229 Ellert Brae',
  'city': 'Mayfield Heights'},)

In [16]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,23,Sports Coach,(195) 714-0885,2306 7790 5218 6927,07/24,Berry Jackson,229 Ellert Brae,Mayfield Heights
1,66,Physicist,807.452.0962,5534 8118 4456 3001,03/25,Keturah English,930 Belles Plaza,Cleveland Heights
2,24,Jockey,(201) 497-2231,4102 8230 2422 5132,05/21,Marguerita Bruce,163 Vicente Private,Taylor
3,18,Art Critic,173-388-1767,2297 1375 3245 1349,01/19,Paz Phillips,1264 Edinburgh Row,Oak Harbor
4,36,Pharmacist,(416) 253-0182,4725 0803 2541 8521,02/25,Mark Baldwin,21 Chula Trace,West Valley City


We can now perform the same computation as before, but now using Pandas and Dask dataframe.

In [17]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

Haulage Contractor      17
Sports Administrator    16
Plastics Engineer       15
Lorry Driver            15
Landworker              14
Garden Designer         14
Ticket Inspector        13
Caulker                 13
Pawnbroker              13
Television Director     13
Name: occupation, dtype: int64

## Learn More

You may be interested in the following links:

-  [Dask Bag Documentation](http://docs.dask.org/en/latest/bag-overview.html)
-  [API Documentation](http://docs.dask.org/en/latest/bag-api.html)
-  [dask tutorial](https://github.com/dask/dask-tutorial), notebook 02, for a more in-depth introduction.

In [18]:
client.close()

In [20]:
%reset -s -f